In [9]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (4,3)
import seaborn as sns
import pandas as pd

%matplotlib inline

## Experiment design

In this game participants needed to make a choice between two options where one was a sure option and another one a gamble with different probabilities ($p$). Game was provided in the context of  5 binary choices under 2 framings. One was a Positive framing, where option A was winning bigger amount of money or getting nothing with particular probabilities  and option B was winning lesser amount of money, but for sure. Second was a Negative framing, where option A was losing bigger amount of money or loosing nothing with particular probabilities  and option B was loosing lesser amount of money, but for sure. Probabilities were $0.1, 0.3, 0.5, 0.7, \text{and } 0.9$. with all options for each probability. All games were presented in random order, and participants filled them twice, ones in placebo condition and once in testosterone condition.

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/iknyazeva/RiskTestosterone/master/Framing.csv',sep = ';')
df.head(10)

,ID,T_W_0.9,PL_W_0.9,T_W_0.7,PL_W_0.7,T_W_0.5,PL_W_0.5,T_W_0.3,PL_W_0.3,T_W_0.1,PL_W_0.1,T_L_0.9,P_L_0.9,T_L_0.7,PL_L_0.7,T_L_0.5,PL_L_0.5,T_L_0.3,P_L_0.3,T_L_0.1,P_L_0.1
0,PK01,2,1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1,1,2
1,PK02,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,2,2,1,2,2
2,PK03,2,1,1,1,1,1,1,1,1,1,1,2,1,2,1,2,1,2,1,2
3,PK04,2,1,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
4,PK05,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1
5,PK06,1,2,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,2,2,2
6,PK07,1,2,2,2,1,2,1,1,1,1,1,1,1,1,2,2,2,2,2,2
7,PK08,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
8,PK09,1,1,2,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,PK10,2,2,2,2,1,2,1,2,1,1,1,1,1,1,1,2,2,2,2,2


In [5]:
m_df = pd.melt(df, id_vars='ID', value_vars=list(df.columns[1:]), value_name='Strategy')
m_df['Test'] = 1
m_df.loc[m_df['variable'].str.contains('P'),'Test'] = 0
m_df['Win'] = 1
m_df.loc[m_df['variable'].str.contains('_L'),'Win'] = 0
m_df['P'] = m_df.apply(lambda row: float(row.variable[-3:]), axis=1)
m_df.drop("variable", axis=1, inplace = True)
m_df.Strategy = m_df.Strategy - 1
win_money = {0.9: {'A': 2000, 'B': 1500}, 0.7: {'A': 500, 'B': 250}, 0.5: {'A': 1000, 'B': 500}, 
             0.3: {'A': 1500, 'B': 500}, 0.1: {'A': 250, 'B': 50}}
loss_money = {0.9: {'A': 2400, 'B': 1200}, 0.7: {'A': 600, 'B': 200}, 0.5: {'A': 1000, 'B': 500}, 
             0.3: {'A': 1800, 'B': 600}, 0.1: {'A': 1200, 'B': 400}}

m_df['A'] = 0
m_df['B'] = 1
m_df.loc[m_df['Win']== 1,'A'] = m_df.loc[m_df['Win']== 1].apply(lambda row: win_money[row.P]['A'], axis=1)
m_df.loc[m_df['Win']== 1,'B'] = m_df.loc[m_df['Win']== 1].apply(lambda row: win_money[row.P]['B'], axis=1)
m_df.loc[m_df['Win']== 0,'A'] = m_df.loc[m_df['Win']== 0].apply(lambda row: loss_money[row.P]['A'], axis=1)
m_df.loc[m_df['Win']== 0,'B'] = m_df.loc[m_df['Win']== 0].apply(lambda row: loss_money[row.P]['B'], axis=1)

m_df["Inds"] = pd.Categorical(m_df['ID']).codes
print(f'Number of inds: {len(pd.unique(m_df.Inds))}')
m_df.loc[m_df['Win']== 0,['A','B']] = -1*m_df.loc[m_df['Win']== 0,['A','B']]
m_df['GameId'] = 0    
grouped = m_df.groupby(['Win','P','Test'])
key_list = list(grouped.groups.keys())
for i,key in zip(range(len(key_list)),key_list):
    m_df.loc[grouped.groups[key_list[i]], 'GameId']  = i
    
m_df['TypeCode'] = 0
hormones = [0, 1, 0, 1]
win =  [1, 1, 0, 0]
codes = [0,1,2,3]
for c,h,w in zip(codes,hormones, win):
    m_df.loc[(m_df['Test']== h)&(m_df['Win']== w),'TypeCode']= c   
m_df.head()
m_df

Number of inds: 40


,ID,Strategy,Test,Win,P,A,B,Inds,GameId,TypeCode
0,PK01,1,1,1,0.9,2000,1500,24,19,1
1,PK02,1,1,1,0.9,2000,1500,25,19,1
2,PK03,1,1,1,0.9,2000,1500,26,19,1
3,PK04,1,1,1,0.9,2000,1500,27,19,1
4,PK05,0,1,1,0.9,2000,1500,28,19,1
...,...,...,...,...,...,...,...,...,...,...
795,MR21,1,0,0,0.1,-1200,-400,19,0,2
796,MR23,1,0,0,0.1,-1200,-400,20,0,2
797,MR24,0,0,0,0.1,-1200,-400,21,0,2
798,MR25,1,0,0,0.1,-1200,-400,22,0,2


## Cumulative prospect theory

According  to prospect theory  (Fox, C. R. & Poldrack, R. A. Prospect Theory 149 Applications to Riskless Choice 151)  choice between two gambles is based on subjective value that represents  its desirability to decision maker. The value $V$ for simple prospect  that payoff $x$  with probability $p$  is given by  $V(x,p)=\omega(x)v(x)$ ,  where $w(x)$ measures impact effect of probability and $v(x)$ is subjective value of money.  The subjective value of payoff is  defined as power function from payoff 

$$V(A) = \sum v(x_i)*\omega(p_i)$$

where $V(A)$ - value function, $\omega(p_i)$ - decision weights for for gains and losses


$$ v(x_i) = \begin{cases} 
    	x^{\alpha}, 
    	& \text{if}\, (x)>0 \\
    	-\lambda (-x)^{\beta}, & \text{else}
    \end{cases}$$
$$ \omega(p) = \frac{p^{c_{ind}}}{[p^{c_{ind}} + (1-p)^{c_{ind}}]^{1/{c_{ind}}}} $$
where $ind = +,\text{ for gain }, - \text{ loss }$

With accordance to prospect theory the decision maker should always choose an option with a larger subjective value. A 0 to 1 choice probability is usually described by exponential transform, which leads to logistic function choice function:

$$p(A,B)  = \frac{1}{1+e^{\phi(V(B)-V(A))}}$$
A - Game, B - sure, in our case


## Testosterone application
These patients underwent the same task twice, one time after 100mg of a colorless, odorless testosterone gel was applied to the shoulders and lower neck and allowed to dry, and one time after a similar-looking placebo gel was applied. The testosterone gel consisted of Testotop®, which is the active testosterone ingredient, 96 percents ethanol which acts as a solvent, polyacrylate and propylene glycol, which together help control the gel’s viscosity, disodium EDTA, trometamol, and purified water. The placebo gel contained the exact same ingredients, excluding the active testosterone ingredient. Participants had a break of at least one week between the two trials; for example, if one participant received the testosterone gel first and performed the task, in one week they would perform the same task but under the placebo condition.  The design of the study was double-blind, meaning that neither the researcher nor the participant was aware of when the participant was being administered the testosterone or placebo condition.  This was evaluated by an additional step in the procedure, in which at the end of the session, the participant and experimenter were asked which treatment they thought the participant had received. Additionally, half of the patients received the testosterone gel first and placebo second, while the other half received the placebo first and testosterone second.

In [6]:
h_df = pd.read_csv('https://raw.githubusercontent.com/iknyazeva/RiskTestosterone/master/horm_sort.csv', usecols = [0,1,2,3,4])

In [7]:
h_df.head()

,Subjects,Time,Type,Testosterone,Cortisol
0,MR01,0,0,12.00,303.60
1,MR02,0,0,15.70,394.70
2,MR03,0,0,16.50,560.30
3,MR04,0,0,14.60,601.70
4,MR05,0,0,28.70,532.70


In [8]:
## Что тут можно нарисовать и какие выводы сделать?

##  Задание. Подумать какая тут нужна визуализация и для чего? Что можно пытаться понять из данных?